**Distributed and Parallel Computing (CS576)**
<br>
Date: **26 February 2020**
<br>
Location: **SU, NEW STEM building**
<br>
Room: **302**
<br>
Week 1
<br>
Title: **Introduction to Parallel Programming**
<br>
Speaker: **Dr. Shota Tsiskaridze**
<br>
Bibliography: [1] Peter S. Pacheco, *An Introduction to Parallel Programming*, Morgan Kaufmann, 2011.


<h1 align="center">Introduction to Parallel Computing</h1>

<h3 align="center">Why Parallel Computing?</h3>

**1) Aren’t single processor systems fast enough?**

As the computational power increases, the number of problems that can be solved also increases.
<br>
Also, by some estimates, the quantity of data stored worldwide **doubles** every two years!

**2) Why can’t microprocessor manufacturers continue to develop much faster single processor systems?**

Air-cooled integrated circuits are reaching the limits of their ability to dissipate heat.

**3) Why build parallel systems and systems with multiple processors?**

If the industry doesn’t continue to bring out new products, it will effectively cease to exist.
**4) Why do not write programs that will automatically convert $serial$ $programs$ into $parallel$ $programs$?**

The bad news is that researchers have had very limited success writing the $translation$ $program$ that convert serial programs into parallel programs.

<h3 align="center">Moore's Law</h3>

- From 1986 to 2002 the performance of microprocessors increased, on average, 50% per year.
- Since 2002 single-processor performance improvement has slowed to about 20% per year.
- The difference in performance increase is related with the multiprocessor systems.
  <img src="images/001_Moores_Law.jpg" width="600" height="600" alt="Moores Law"  align="center"/>

<h3 align="center">Example 1</h3>

Suppose that we need to compute $n$ values and add them together. For example $n = 24$:

$$1, 4, 3, 9, 2, 8, 5, 1, 1, 6, 2, 7, 2, 5, 0, 4, 1, 8, 6, 5, 1, 2, 3, 9.$$

We know that this can be done with the following serial code (Pseudo-code):

In [ ]:
sum = 0
for i in range(0, n):
    x = "Compute next value"
    sum += x

Now, if we have $p=8$ cores, then each core can sum of approximately $n/p=3$ values:

|  Core  | 0 | 1 | 2 | 3 | 4 | 5 | 6 | 7 |
|--------|---|---|---|---|---|---|---|---|
| my_sum | 8 | 19| 7 | 15| 7 | 13| 12| 14|

This can be done with the following serial code (Pseudo-code):

In [ ]:
my_sum = 0
my_first_i = ...
my_last_i = ...
for i in range(my_first_i, my_last_i):
    my_x = "Compute next value"
    my_sum += my_x

In [ ]:
if "I’m the master core":
    sum = my x
    for "each core other than myself":
        "receive value from_core"
        sum += value
else:
    "send my_x to the master"

When the cores are done computing their values of $my$ $sum$, they send their results to a designated $master$ core, which can form a $global$ $sum$. In our example, if the master core is $core$ $0$, it would add the values 8+19+7+15+7+13+12+14=95.

We can probably see another way to do this: instead of making the master core do all the work of computing the final sum, we can $pair$ the cores as shown below:

<img src="images/Figure_1.png" width="800" height="800" alt="Example"  align="center"/>

With $p=1000$ cores, the first method will require $999$ receives and adds, while the second will only require $10$, an improvement of almost a factor of $100!$

<h3 align="center">Conclusions</h3>

- The **first global sum** is a fairly obvious generalization of the **serial global sum**.
- The **second global sum**, on the other hand, bears little relation to the original serial addition.

- The point here is that it’s unlikely that a **translation program** would “discover” the second global sum.

- Thus, we cannot simply continue to write serial programs, we must write parallel programs!

<h3 align="center">How to Write Parallel Programs?</h3>

$\bullet$ There are a number of possible answers to this question, but most of them depend on the basic idea of <br>
&ensp; $partitioning$ the work to be done among the cores. 


$\bullet$ There are two widely used approaches:
<br>
&emsp; $\bullet$ **task-parallelism**: partitioning the various tasks carried out in solving the problem among the cores.
<br>
&emsp; $\bullet$ **data-parallelism**: partitioning the data used in solving the problem among the cores.

<h3 align="center">Example 2</h3>

Suppose that **Prof. E** has to teach a section **"Distributed and Parallel Computing"**.
<br>
Also suppose that **Prof. E** has **thirty students** in his section, 
<br>
so he has been assigned a teaching assistants (TAs): **Dr. K** and **Dr. S**.
<br>
At last the semester is over, and **Prof. E** makes up a final exam that consists of **three questions**.
<br>
In order to grade the exam, **Prof. E** and his TAs might consider the following two options:
- each of them can grade all thirty responses to one of the questions. <br> Say **Prof. P** grades question **1**, **Dr. K** grades question **2** and **Dr. S** grades question **3**;
- they divide the **thirty** exams into **three piles** of **ten exams** each, <br>and each of them can grade all the papers in one of the piles.

In both approaches the **cores** are the **Prof. E** and his TAs (**Dr. K** and **Dr. S**).
<br>
- The **first** approach might be considered an example of **task-parallelism**
- The **second** approach might be considered an example of **data-parallelism**

<h3 align="center">Coordination of the Cores</h3>

When we write parallel programs, we usually need to **coordinate** the work of the cores.
<br>
Let's show this on Example 1. In both global sum examples coordination involves:
<br>
&emsp; $\bullet$ **communication** among the cores: one or more cores send their current partial sums to another core;
<br>
&emsp; $\bullet$ **load balancing**: all cores are assigned roughly the same number of values to compute;

A third type of coordination is **synchronization**.
<br>In most systems the cores are not automatically synchronized: each core works at its own pace.
<br>
Suppose that we need to **read data** from file. Say $x$ is an array that is read in by the **master** core:

In [ ]:
if "I’m the master core":
    fi = open('path/to/file.txt', 'r')
    for i in range (0, n):
        line = fi.readline()
        x[i] = line

We don’t want the other cores to race ahead and start computing their partial sums before the master is done initializing $x$.
Therefore, We need to add in a point of synchronization between the initialization of $x$ and the computation of the partial sums: *Synchronize cores()*

<h3 align="center">Shared and Distributed Memory</h3>

There are two main types of parallel systems that we’ll be focusing on:
<br>
&emsp; $\bullet$ **shared-Memory** systems (left): the cores share access to the computer’s memory;
<br>
&emsp; $\bullet$ **distributed-Memory** systems (right): each core has its own (private) memory.
<br>
&emsp; The cores communicate explicitly by sending messages across a network.


<img src="images/Memory_Organization.png" width="1200" height="80" alt="Example"  align="center"/>


<h3 align="center">Concurrent, Parallel and Distributed Computings</h3>

Although there isn’t complete agreement on the distinction between the terms parallel, distributed, and concurrent, many authors make the following distinctions:

- **parallel computing**: runs multiple tasks simultaneously on cores that are physically close to each other and that either share the same memory or are connected by a very high-speed network;

- **distributed computing**, tend to be more “loosely coupled.” The tasks may be executed by multiple computers that are separated by large distances, and the tasks themselves are often executed by programs that were created independently.

- **concurrent computing**: multiple tasks can be in progress at any instant. For example, multitasking operating system is also concurrent, even when it is run on a machine with only one core, since multiple tasks can be in progress at any instant


**Parallel hardware and software** have grown out of conventional **serial hardware and software**

In order to better understand the current state of parallel systems, let’s begin with a brief look at a few aspects of **serial systems**.

<h3 align="center">The von Neumann architecture</h3>

The *classical* von Neumann architecture consists of: 
$$\textbf{Main memory, Central Processing Unit (CPU) and Interconnection}.$$

<img src="images/Von_Neumann_Model.png" width="500" height="500" alt="Example"  align="center"/>


- **Main memory** consists of a collection of **locations**, each of which is capable of storing both **instructions** and **data**:
 - Every **location** consists of an **address**, which is used to access the location and the **contents** of the location — the instructions or data stored in the location.

- **CPU** is divided into a **control unit** and an **Arithmetic and Logic Unit (ALU)**:
 - The **control unit** is responsible for deciding **which** instructions in a program should be executed;
 - The **ALU** is responsible for executing the actual instructions.

- **Interconections** is used to transfer instructions between the CPU and memory. This has traditionally been a **bus**, which consists of a collection of parallel wires and some hardware controlling access to the wires.
 - When data/instructions are transferred from memory to the CPU, we say the data/instructions are **read** from memory. 
 - When data are transferred from the CPU to memory, we  say the data are **written** to memory.



<h3 align="center">The von Neumann Bottleneck</h3>

- In 2010 CPUs are capable of executing instructions more than **one hundred times** faster than they can read items from main memory. 
- This rises the so called the **Von Neuman Bottleneck Problem**.
- Led to the **Modifications to the Von Neuman Model**.

<img src="images/Factory.png" width="1800" height="600" alt="Example"  align="center"/>

<h3 align="center">Processes, Multitasking, and Threads</h3>

$\bullet$ **Operation System (OS)**: the most important software on a computer. 
<br>
&emsp; It manages the computer’s hardware and software resources.
<br>
$\bullet$ **Multitasking**: most modern OS supports the simultaneous execution of **multiple programs**.
<br>
&emsp; This is **possible** even on a system with a **single core**.
<br>
$\bullet$ **Process** - an instance of a computer program that is being executed. A process consists of:
<br>
&emsp; $\bullet$ The executable machine language program.
<br>
&emsp; $\bullet$ A block of memory, which will include the executable code.
<br>
&emsp; $\bullet$ Descriptors of resources that the operating system has allocated to the process.
<br>
&emsp; $\bullet$ Security information specifying which hardware and software resources can be used.
<br>
&emsp; $\bullet$ Information about the state of the process (ready, registry, process' memory).
<br>
$\bullet$ **Threading**: a mechanism to divide the process into more or less independent tasks.
<br>
&emsp; A **threads** can be stopped and started **much faster** than **processes**.
<img src="images/Process.png" width="800" height="200" alt="Example"  align="center"/>


<h3 align="center">Modification to the Von Neumann Model: CPU Caching</h3>


$\bullet$ **CPU cache** is a collection of memory locations that CPU can access more quickly than main memory.
<br>
$\bullet$ **Caching** is one of the most widely used methods of addressing the von Neumann bottleneck:
<br>
&emsp; $\bullet$ **widen** the road, i.e. transport more data or more instructions in a single memory access.
<br>
&emsp; $\bullet$ **move** the **factory/warehouse**, i.e. store blocks of data and instructions in special memory, 
<br> 
&emsp; &emsp; that is effectively closer to the registers in the CPU.
<br> 
$\bullet$ **Locality**: the principle that an access of one location is followed by an access of a nearby location.
<br> 
$\bullet$ **Cache blocks (or cache lines)**: stores **8 to 16 times** as much information as a single memory.
<br> 
$\bullet$ **Cache levels**: the first level (**L1**) is the smallest**&**fastest, and higher levels (**L2**, . . . ) are larger**&**slower.
<br> 
$\bullet$ **Cache hit**: when a cache is checked for information and the information is available.
<br> 
$\bullet$ **Cache miss**: when a cache is checked for information and the information is unavailable.
<br> 
$\bullet$ **Inconsistecy**: the value in the cache and the value in main memory are different.
<br>
&emsp; $\bullet$ **In write-through caches**: the line is written to main memory when it is written to the cache.
<br>
&emsp; $\bullet$ **In write-back caches**: the line is written to main memory when the cache line is replaced by a new one.




<h3 align="center">Modification to the Von Neumann Model: Cache Mapping</h3>


$\bullet$ Another issue in cache design is deciding **where lines should be stored**:
<br>
&emsp; $\bullet$ **fully associative cache**: a new line can be placed at any location in the cache.
<br>
&emsp; $\bullet$ **direct mapped cahce**: each cache line has a unique location in the cache to which it will be assigned.
<br>
&emsp; $\bullet$ $n$-**way set associative.**: each cache line can be placed in one of $n$ different locations in the cache.


Suppose our **main memory** consists of $8$ lines and our **cache** consists of $4$ lines:

|  Memory Index  | Fully Associative | Direct Mapped| 2-Way Set Associative|
|----------------|-------------------|--------------|----------------------|
|        0       | 0, 1, 2, or 3     |      0       |     0 or 1           |
|        1       | 0, 1, 2, or 3     |      1       |     2 or 3           |
|        2       | 0, 1, 2, or 3     |      2       |     0 or 1           |
|        3       | 0, 1, 2, or 3     |      3       |     2 or 3           |
|        4       | 0, 1, 2, or 3     |      0       |     0 or 1           |
|        5       | 0, 1, 2, or 3     |      1       |     2 or 3           |
|        6       | 0, 1, 2, or 3     |      2       |     0 or 1           |
|        7       | 0, 1, 2, or 3     |      3       |     2 or 3           |



<h3 align="center">Example</h3>


$\bullet$ **Note:** the workings of the CPU cache are **controlled by the system hardware**,
<br>
$\bullet$ However, knowing the **principle of locality allows** us to have some indirect control over caching.

$\bullet$ Suppose that we need to **sum up** the data in the **two-dimensional array** and 
<br>
$\bullet$ let’s also suppose that cache is **direct mapped** and it can only store $8$ elements, or $2$ **cache lines**
.

In [ ]:
A[n][n], x[n], y[n]

# First pair of loops
A[n][n], x[n], y[n]
for i in range(0,n):
    for j in range(0,n):
        y[i] += A[i][j]*x[j]

# Second pair of loops
for j in range(0,n):
    for i in range(0,n):
        y[i] += A[i][j]*x[j]

$\bullet$ First pair of loops results in 2 misses while the second pair of loops results in 16 misses.

<h3 align="center">Modification to the Von Neumann Model: Virtual Memory</h3>

$\bullet$ **Issues with main memory**: 
<br>
&emsp; $\bullet$ all of the instructions and data may not fit into **main memory** when we run **very large programs**.
<br>
&emsp; $\bullet$ data and instructions must be protected while sharing the **main memory** among the running programs.

$\bullet$ **Virtual memory** was developed so that main memory can function as a cache for secondary storage. It:
<br>
&emsp;  $\bullet$ operates on relatively large blocks (from $4$ to $16$ kilobytes) of data and instructions, called **pages**.
<br>
&emsp;  $\bullet$ uses **virtual addresses** instead of addressing the memory used by a program with physical addresses.
<br> 
&emsp;  $\bullet$ uses **page table** to translate the virtual address into a physical address.
<img src="images/Virtual_Memory.jpg" width="700" height="400" alt="Example"  align="center"/>

<h3 align="center">Modification to the Von Neumann Model: Instruction-Level Parallelism</h3>

$\bullet$ **Instruction-Level Parallelism (ILP)** is used to improve processor performance by having multiple processor components or **functional units** simultaneously executing instructions.
There are two main approaches to ILP:
<br>
&emsp; $\bullet$ **pipelining**, in which functional units are arranged in stages.
<br>
&emsp; $\bullet$ **multiple issue**, in which multiple instructions can be simultaneously initiated.
<br>
$\bullet$ Both approaches are used in virtually all modern CPUs.

$\bullet$ In general, **ILP** can be **very difficult to exploit**: it is a program with a long sequence of dependent statements offers few opportunities.
<br>
$\bullet$ For example, in a direct calculation of the **Fibonacci numbers** there’s essentially no opportunity for simultaneous execution of instructions:

In [ ]:
F[0] = F[1] = 1
for i in range(2, n):
  F[i] = F[i-1] + F[i-2]

<h3 align="center">Pipelining</h3>

$\bullet$ The principle of pipelining is similar to a **factory assembly line**:
<br>
&emsp; $\bullet$ the **first team** is bolting a car’s engine to the chassis.
<br>
&emsp; $\bullet$ the **second team** can connect the transmission to the engine and the driveshaft of a car,
<br>
&emsp; &ensp; that’s already been processed by the first team.
<br>
&emsp; $\bullet$ the **third team** can bolt the body to the chassis in a car that’s been processed by the first two teams.

<h3 align="center">Example</h3>

$\bullet$ Suppose we want to add two floating point numbers $3.14 \cdot 10^{4}$ and $2.72 \cdot 10^{3}$. The steps will be:

<font size="5">

| Time  | Operation         | Operand 1            | Operand 2            | Result                  |
|-------|-------------------|----------------------|----------------------|-------------------------|
|   0   | Fetch operands    | $9.99 \times 10^{9}$ | $8.88 \times 10^{8}$ |                         |
|   1   | Compare exponents | $9.99 \times 10^{9}$ | $8.88 \times 10^{8}$ |                         |
|   2   | Shift one operand | $9.99 \times 10^{9}$ | $8.88 \times 10^{8}$ |                         |
|   3   | Add               | $9.99 \times 10^{9}$ | $8.88 \times 10^{8}$ | $10.878 \times 10^{9}$  |
|   4   | Normalize result  | $9.99 \times 10^{9}$ | $8.88 \times 10^{8}$ | $1.0878 \times 10^{10}$ |
|   5   | Round result      | $9.99 \times 10^{9}$ | $8.88 \times 10^{8}$ | $1.09 \times 10^{10}$   |
|   6   | Store result      | $9.99 \times 10^{9}$ | $8.88 \times 10^{8}$ | $1.0878 \times 10^{10}$ |
|-------|-------------------|----------------------|----------------------|-------------------------|

</font>

$\bullet$ If each of the operations takes one nanosecond, the addition operation will take 7 nanoseconds
<br>
$\bullet$ So if we execute the code:

In [ ]:
x[1000], y[1000], z[1000]
...
for i in range(0, 1000):
    z[i] = x[i] + y[i]

the **for** loop will take something like 7000 nanoseconds.

$\bullet$ Alternative  is to divide our floating point adder into seven separate pieces and use  the **pipelining**:

| Time  | Fetch | Compare | Shift | Add | Normalize | Roound | Store |
|-------|-------|---------|-------|-----|-----------|--------|-------|
| 0     | 0     |         |       |     |           |        |       |
| 1     | 1     | 0       |       |     |           |        |       |
| 2     | 2     | 1       | 0     |     |           |        |       |
| 3     | 3     | 2       | 1     | 0   |           |        |       |
| 4     | 4     | 3       | 2     | 1   | 0         |        |       |
| 5     | 5     | 4       | 3     | 2   | 1         | 0      |       |
| 6     | 6     | 5       | 4     | 3   | 2         | 1      | 0     |
| ...   | ...   | ...     | ...   | ... | ...       | ...    | ...   |
| 999   | 999   | 998     | 997   | 996 | 995       | 994    | 993   |
| 1000  |       | 999     | 998   | 997 | 996       | 995    | 994   |
| 1001  |       |         | 999   | 998 | 997       | 996    | 995   |
| 1002  |       |         |       | 999 | 998       | 997    | 996   |
| 1003  |       |         |       |     | 999       | 998    | 997   |
| 1004  |       |         |       |     |           | 999    | 998   |
| 1005  |       |         |       |     |           |        | 999   |

<h3 align="center">Multiple Issue</h3>

$\bullet$ **Multiple issue** processors replicate functional units and try to simultaneously execute different instructions in a program.
<br>
$\bullet$ For example, we can approximately **halve the time** of the **floating point adders** to execute the loop: 
<br>
&emsp; $\bullet$  while the first adder is computing $z[0]$, the second can compute $z[1]$; 
<br> 
&emsp; $\bullet$  while the first is computing $z[2]$, the second can compute $z[3]$; and so on.
<br>
$\bullet$ **Static** multiple issue: the functional units are scheduled at compile time
<br>
$\bullet$ **Dynamic** multiple issue: the functional units scheduled at run-time.

$\bullet$ A **processor** that supports **dynamic** multiple issue is sometimes said to be **superscalar**.
<br>
$\bullet$ The most important techniques to find instructions that can be executed simultaneously is **speculation**:
<br>
&ensp; the processor makes a guess about an instruction, then executes the instruction on the basis of the guess.

In [ ]:
z = x + y
if (z > 0): w = x
    
else:
    w = y

<h3 align="center">Modification to the Von Neumann Model: Thread-Level Parallelism</h3>

$\bullet$ **Thread-level parallelism (TLP)** attempts to provide parallelism through the simultaneous execution of different threads, i.e. it's more **coarser-grained** parallelism rather than the **finer-grained** parallelism.

$\bullet$ In **fine-grained** multithreading, the processor switches between threads after each instruction, skipping threads that are stalled. While this approach has the potential to avoid wasted machine time due to stalls, it has the drawback that a thread that’s ready to execute a long sequence of instructions may have to wait to execute every instruction.

$\bullet$ **Coarse-grained** multithreading attempts to avoid this problem by only switching threads that are stalled waiting for a time-consuming operation to complete.

$\bullet$  In order for **TLP** to be useful, the system must support **very rapid switching** between threads.

$\bullet$ **Simultaneous multithreading (SMT)** is a variation on fine-grained multithreading. It attempts to exploit superscalar processors by allowing multiple threads.